# Лабораторная работа: Мета-обучение
## Шаг 1: Сбор данных с OpenML

**Цель:** Собрать минимум 300 датасетов для мета-обучения

In [ ]:
import sys
sys.path.append('..')

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import openml
from pathlib import Path

from src.data.collector import OpenMLCollector
from src.data.filters import DatasetFilter
from src.visualization.plots import MetaVisualizer

%matplotlib inline
sns.set_style('whitegrid')
plt.rcParams['figure.figsize'] = (12, 8)

In [ ]:
# Инициализация
collector = OpenMLCollector(cache_dir='../data/cache/openml_cache')
visualizer = MetaVisualizer(save_dir='../results/visualizations')

print("OpenML версия:", openml.__version__)

### 1.1 Получение списка датасетов с фильтрацией

In [ ]:
datasets_df = collector.fetch_datasets_list(
    min_instances=100,      # минимум объектов
    max_instances=10000,    # максимум объектов
    min_features=3,          # минимум признаков
    max_features=500,        # максимум признаков
    min_classes=2,           # минимум классов
    limit=350                # ограничение количества
)

print(f"Найдено {len(datasets_df)} датасетов после фильтрации")
print(f"Диапазон объектов: {datasets_df['NumberOfInstances'].min()} - {datasets_df['NumberOfInstances'].max()}")
print(f"Диапазон признаков: {datasets_df['NumberOfFeatures'].min()} - {datasets_df['NumberOfFeatures'].max()}")
print(f"Диапазон классов: {datasets_df['NumberOfClasses'].min()} - {datasets_df['NumberOfClasses'].max()}")

In [ ]:
# Покажем первые 10 датасетов
datasets_df[['did', 'name', 'NumberOfInstances', 'NumberOfFeatures', 'NumberOfClasses']].head(10)

### 1.2 Визуализация распределений

In [ ]:
fig, axes = plt.subplots(2, 3, figsize=(15, 10))

# 1. Количество объектов
axes[0, 0].hist(datasets_df['NumberOfInstances'], bins=30, edgecolor='black', alpha=0.7)
axes[0, 0].set_xlabel('Количество объектов')
axes[0, 0].set_ylabel('Частота')
axes[0, 0].set_title('Распределение по размеру датасетов')

# 2. Количество признаков
axes[0, 1].hist(datasets_df['NumberOfFeatures'], bins=30, edgecolor='black', alpha=0.7, color='orange')
axes[0, 1].set_xlabel('Количество признаков')
axes[0, 1].set_ylabel('Частота')
axes[0, 1].set_title('Распределение по количеству признаков')

# 3. Количество классов
axes[0, 2].hist(datasets_df['NumberOfClasses'], bins=15, edgecolor='black', alpha=0.7, color='green')
axes[0, 2].set_xlabel('Количество классов')
axes[0, 2].set_ylabel('Частота')
axes[0, 2].set_title('Распределение по количеству классов')

# 4. Объекты vs Признаки (scatter)
axes[1, 0].scatter(datasets_df['NumberOfInstances'], datasets_df['NumberOfFeatures'], alpha=0.6)
axes[1, 0].set_xlabel('Количество объектов')
axes[1, 0].set_ylabel('Количество признаков')
axes[1, 0].set_title('Объекты vs Признаки')
axes[1, 0].set_xscale('log')
axes[1, 0].set_yscale('log')

# 5. Размерность (признаки/объекты)
dimensionality = datasets_df['NumberOfFeatures'] / datasets_df['NumberOfInstances']
axes[1, 1].hist(dimensionality, bins=30, edgecolor='black', alpha=0.7, color='red')
axes[1, 1].set_xlabel('Размерность (признаки/объекты)')
axes[1, 1].set_ylabel('Частота')
axes[1, 1].set_title('Распределение размерности')

# 6. Информация
axes[1, 2].text(0.1, 0.5, 
                f'Всего датасетов: {len(datasets_df)}\n\n'
                f'Средний размер: {datasets_df["NumberOfInstances"].mean():.0f}\n'
                f'Среднее число признаков: {datasets_df["NumberOfFeatures"].mean():.1f}\n'
                f'Среднее число классов: {datasets_df["NumberOfClasses"].mean():.1f}',
                fontsize=12, va='center')
axes[1, 2].axis('off')

plt.suptitle('Анализ собранных датасетов', fontsize=16)
plt.tight_layout()
plt.show()

### 1.3 Сохранение списка датасетов

In [ ]:
datasets_df.to_csv('../data/raw/dataset_list.csv', index=False)
print("Список датасетов сохранен в ../data/raw/dataset_list.csv")

# Проверка
saved_df = pd.read_csv('../data/raw/dataset_list.csv')
print(f"Загружено {len(saved_df)} датасетов из сохраненного файла")

### 1.4 Выводы по шагу 1

In [ ]:
print("="*60)
print("ВЫВОДЫ ПО ШАГУ 1: СБОР ДАННЫХ")
print("="*60)
print(f"1. Собрано {len(datasets_df)} датасетов, что достаточно для мета-обучения (>300)")
print(f"2. Диапазон размеров: {datasets_df['NumberOfInstances'].min()}-{datasets_df['NumberOfInstances'].max()} объектов")
print(f"3. Диапазон признаков: {datasets_df['NumberOfFeatures'].min()}-{datasets_df['NumberOfFeatures'].max()}")
print(f"4. Диапазон классов: {datasets_df['NumberOfClasses'].min()}-{datasets_df['NumberOfClasses'].max()}")
print(f"5. Данные сохранены для дальнейшей обработки")